# Generadores Congruenciales Lineales

Los **Generadores Congruenciales Lineales** (GCL) son una familia de algoritmos para generar secuencias de números pseudoaleatorias. Se basan en una relación de recurrencia simple. 

## Definición

Los GCL se basan en una combinación lineal de los últimos $k$ enteros generados y se calcula su resto al dividir por un entero fijo $m$. En el método congruencial simple (de orden $k = 1$), partiendo de una semilla inicial $X_0$, el algoritmo se basa en la siguiente recurrencia matemática:

$$X_{n+1} = (aX_n + c) \bmod m$$

donde:
- $X_n$: es el valor actual de la secuencia
- $a$: es el multiplicador
- $c$: es el incremento 
- $m$: es el módulo
- $X_{n+1}$: siguiente número pseudoaleatorio de la secuencia

## Tipos de Generadores
- Si $c = 0$: *congruencial multiplicativo* (o *congruencial de Lehmer*)
- Si $c \neq 0$: *congruencial mixto*

## Normalización

Para obtener valores en el intervalo $[0,1]$, se calcula:

$$U_n = \frac{X_n}{m}$$

## Ventajas
- *Simplicidad*: Fáciles de implementar y entender
- Con la misma configuración de parámetros ($a,c,m$), misma semilla produce misma secuencia

## Desventajas
- *Período limitado*: $\text{Máximo período} = m$
- *Parámetros críticos*: Requieren parámetros cuidadosamente elegidos 

In [17]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from typing import List, Tuple
import seaborn as sns 

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## Implementación Básica del Generador Congruencial Lineal

In [18]:
class LinearCongruentialGenerator:
  """Implementación de un Generador Congruencial Lineal (GCL). 
  
  Con fórmula base: $X_{n+1} = (a * X_n + c) mod m$
  
  Parámetros:
  -----------
  a : int 
      Multiplicador (debe ser mayor que 0)
  c : int 
      Incremento (si c=0, es un generador multiplicativo)
  m : int 
      Módulo (entero positivo)
  seed : int 
      Semilla inicial X_0
  """
  
  def __init__(self, a:int, c:int, m:int, seed:int) -> None:
    self.a = a            # multiplicador
    self.c = c            # incremento
    self.m = m            # módulo
    self.seed = seed      # semilla inicial
    self.current = seed   # valor actual de la secuencia
    self.sequence = [ ]   # almacena la secuencia generada
    
    if a <= 0 or m <= 0:
      raise ValueError("Los parámetros 'a' y 'm' deben ser positivos")
    if c < 0:
      raise ValueError("El parámetro 'c' debe ser no negativo")
    if seed < 0 or seed >= m:
      raise ValueError("La semilla debe estar en el rango [0, m-1]")
  
  def _next(self) -> int:
    """Genera el siguiente número en la secuencia

    Returns:
      int: el siguiente número pseudoaleatorio
    """
    # aplicar la fórmula del GCL 
    self.current = (self.a * self.current + self.c) % self.m 
    self.sequence.append(self.current)
    return self.current
  
  def generate_sequence(self, n:int) -> List[int]:
    """Genera una secuencia de n números pseudoaleatorios. 

    Args:
      n (int): número de elementos a generar 

    Returns:
      List[int]: lista con los n números generados
    """
    sequence = [ ]
    for _ in range(n):
      sequence.append(self._next())
    return sequence
  
  def generate_uniforms(self, n:int) -> List[float]:
    """Genera una secuencia de n números uniformes en [0,1] usando: U_n = X_n / m

    Args:
      n (int): número de elementos a generar

    Returns:
      List[float]: lista con los n números uniformes
    """
    sequence = self.generate_sequence(n)
    return [ x / self.m for x in sequence ]
  
  def reset(self):
    "Reinicia el generador con la semilla originial"
    self.current = self.seed 
    self.sequence = [ ]
  
  def __str__(self) -> str:
    return f"Generador Congruencial Lineal\nParámetros: a={self.a}, c={self.c}, m={self.m}\nSemilla: {self.seed}\nFórmula: X_{{n+1}} = ({self.a} * X_n + {self.c}) mod {self.m}" 
  
  def __repr__(self) -> str:
    return self.__str__()

## Ejemplo 1: Generador Congruencial Mixto

In [19]:
# parámetros para un generador mixto
a = 1664525
c = 1013904223
m = 2**32
seed = 12345

lcg1 = LinearCongruentialGenerator(a, c, m, seed)
print(lcg1)

N = 10

print(f"\nPrimeros {N} valores de la secuencia")
sequence = lcg1.generate_sequence(N)
for i, x_i in enumerate(sequence):
  print(f"X_{i+1} = {x_i}")

lcg1.reset()
print(f"\nPrimeros {N} valores uniformes de la secuencia")
sequence = lcg1.generate_uniforms(N)
for i, x_i in enumerate(sequence):
  print(f"X_{i+1} = {x_i}")

Generador Congruencial Lineal
Parámetros: a=1664525, c=1013904223, m=4294967296
Semilla: 12345
Fórmula: X_{n+1} = (1664525 * X_n + 1013904223) mod 4294967296

Primeros 10 valores de la secuencia
X_1 = 87628868
X_2 = 71072467
X_3 = 2332836374
X_4 = 2726892157
X_5 = 3908547000
X_6 = 483019191
X_7 = 2129828778
X_8 = 2355140353
X_9 = 2560230508
X_10 = 3364893915

Primeros 10 valores uniformes de la secuencia
X_1 = 0.02040268573909998
X_2 = 0.01654784823767841
X_3 = 0.5431557944975793
X_4 = 0.6349040560889989
X_5 = 0.9100295137614012
X_6 = 0.11246166913770139
X_7 = 0.4958894052542746
X_8 = 0.5483488442841917
X_9 = 0.5961001170799136
X_10 = 0.7834504160564393


## Ejemplo 2: Generador Congruencial Multiplicativo (Lehmer)

In [20]:
# parámetros para un generador multiplicativo (Lehmer)
a = 16807
c = 0
m = 2**31 - 1
seed = 12345

lcg2 = LinearCongruentialGenerator(a, c, m, seed)
print(lcg2)

print(f"\nPrimeros {N} valores de la secuencia")
sequence = lcg2.generate_sequence(N)
for i, x_i in enumerate(sequence):
  print(f"X_{i+1} = {x_i}")

lcg2.reset()
print(f"\nPrimeros {N} valores uniformes de la secuencia")
sequence = lcg2.generate_uniforms(N)
for i, x_i in enumerate(sequence):
  print(f"X_{i+1} = {x_i}")

Generador Congruencial Lineal
Parámetros: a=16807, c=0, m=2147483647
Semilla: 12345
Fórmula: X_{n+1} = (16807 * X_n + 0) mod 2147483647

Primeros 10 valores de la secuencia
X_1 = 207482415
X_2 = 1790989824
X_3 = 2035175616
X_4 = 77048696
X_5 = 24794531
X_6 = 109854999
X_7 = 1644515420
X_8 = 1256127050
X_9 = 1963079340
X_10 = 1683198519

Primeros 10 valores uniformes de la secuencia
X_1 = 0.09661652850760917
X_2 = 0.8339946273872604
X_3 = 0.9477024976851895
X_4 = 0.035878594981449935
X_5 = 0.011545853229028104
X_6 = 0.051155220275351417
X_7 = 0.7657871678312249
X_8 = 0.5849297393974521
X_9 = 0.9141300529773021
X_10 = 0.7838003895170057


## Estadísticas 

### Media 

La media teórica de un generador congruencial que produce números pseudoaleatorios uniformemente distribuidos en el intervalo es:

$$\mu = \frac{a + b}{2} = \frac{0 + 1}{2} = 0.5$$

donde $a = 0$ y $b = 1$ son los límites inferior y superior del intervalo. Esta media de 0.5 representa el punto central del intervalo y es el valor esperado teórico para cualquier secuencia de números pseudoaleatorios uniformemente distribuidos.

### Varianza

La varianza teórica para la distribución uniforme continua $U(0,1)$ se calcula mediante la fórmula:

$$\sigma^2 = \frac{(b-a)^2}{12} = \frac{(1 - 0)^2}{12} = \frac{1}{12} \approx 0.0833$$ 

Esta fórmula se deriva de las propiedades fundamentales de la distribución uniforme y representa la medida de dispersión alrededor de la media. La varianza indica qué tan dispersos están los valores generados respecto al valor central de 0.5. 

### Desviación Estándar

La desviación estándar es la raíz cuadrada de la varianza: 

$$\sigma = \sqrt{ \frac{(b-a)^2}{12} } = \frac{ b-a }{ \sqrt{12} } = \frac{1 - 0}{ \sqrt{12} } \approx 0.2887$$

Esta estadística proporciona una medida de la dispersión en las mismas unidades que los datos originales.

In [21]:
# generar números uniformes en [0,1] con ambos generadores
N = 1000 

# reiniciar generadores
lcg1.reset()  # mixto
lcg2.reset()  # lehmer

# generar secuencias uniformes
sequence1 = lcg1.generate_uniforms(N)
sequence2 = lcg2.generate_uniforms(N)

print(f"""=== Estadísticas de Generador Mixto (N={N}) ===
Media: {np.mean(sequence1)} (teórico: 0.5)
Varianza: {np.var(sequence1)} (teórico: 0.0833)
Desviación Estándar: {np.std(sequence1)} (teórico: 0.2887)
Mínimo: {np.min(sequence1)} 
Máximo: {np.max(sequence1)}
""")

print(f"""=== Estadísticas de Generador Lehmer (N={N}) ===
Media: {np.mean(sequence2)} (teórico: 0.5)
Varianza: {np.var(sequence2)} (teórico: 0.0833)
Desviación Estándar: {np.std(sequence2)} (teórico: 0.2887)
Mínimo: {np.min(sequence2)}
Máximo: {np.max(sequence2)}
""")

=== Estadísticas de Generador Mixto (N=1000) ===
Media: 0.5072595152379945 (teórico: 0.5)
Varianza: 0.08173803197303794 (teórico: 0.0833)
Desviación Estándar: 0.28589863933400933 (teórico: 0.2887)
Mínimo: 0.00012929551303386688 
Máximo: 0.9999259654432535

=== Estadísticas de Generador Lehmer (N=1000) ===
Media: 0.4912281235294548 (teórico: 0.5)
Varianza: 0.08206808420920461 (teórico: 0.0833)
Desviación Estándar: 0.2864752767852832 (teórico: 0.2887)
Mínimo: 4.2433384825677324e-05
Máximo: 0.9968053325995828

